In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.datasets import fetch_openml
# Load the dataset (replace 'your_data_id' with the actual data_id)
data = pd.read_csv('/Users/sahilaggarwal/Desktop/Housing.csv')

2024-01-09 00:20:58.524021: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
x=data.drop(columns=['price'])
y=data['price']
y = np.log1p(y) 

In [4]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [5]:
x_train_encoded = pd.get_dummies(x_train)
x_test_encoded = pd.get_dummies(x_test)

In [6]:
x_train_encoded, x_test_encoded = x_train_encoded.align(x_test_encoded, join='left', axis=1, fill_value=0)


In [7]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_encoded)
x_test_scaled = scaler.transform(x_test_encoded)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(x_train_scaled.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Model compilation
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Model training with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(x_train_scaled, y_train, epochs=200, batch_size=32, verbose=1, validation_split=0.2, callbacks=[early_stopping])

# Model evaluation
y_pred = model.predict(x_test_scaled)
y_test = np.expm1(y_test)  # Reverting the normalization
y_pred = np.expm1(y_pred)  # Reverting the normalization
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))


Epoch 1/200
11/11 [==============================] - 0s 10ms/step - loss: 207.9787 - val_loss: 176.7681
Epoch 2/200
11/11 [==============================] - 0s 2ms/step - loss: 156.5345 - val_loss: 123.3703
Epoch 3/200
11/11 [==============================] - 0s 3ms/step - loss: 102.1106 - val_loss: 68.1517
Epoch 4/200
11/11 [==============================] - 0s 2ms/step - loss: 49.7405 - val_loss: 24.0056
Epoch 5/200
11/11 [==============================] - 0s 3ms/step - loss: 14.9563 - val_loss: 5.5517
Epoch 6/200
11/11 [==============================] - 0s 2ms/step - loss: 4.3106 - val_loss: 4.6851
Epoch 7/200
11/11 [==============================] - 0s 2ms/step - loss: 3.2668 - val_loss: 3.5945
Epoch 8/200
11/11 [==============================] - 0s 2ms/step - loss: 1.9353 - val_loss: 2.3351
Epoch 9/200
11/11 [==============================] - 0s 2ms/step - loss: 1.4763 - val_loss: 2.0686
Epoch 10/200
11/11 [==============================] - 0s 2ms/step - loss: 1.3111 - val_loss: 1